In [1]:
%%file guipioneers.py
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 18 11:39:18 2021

@author : greg
@purpose:
@version:
"""

from PyQt5 import uic, QtGui, QtCore, QtWidgets
import time
import pathlib
import os
import sys

from pico8742ctrl import Pico8742Ctrl


MESSAGEDEVICENOTFOUND = "ERROR: Device not found on any USB port."
MESSAGEMOTORALREADYCONNECTED = "WARNING: Motor is already connected"
MESSAGEMOTORPERMISSIONERROR = "ERROR: Device already in use (permission denied)."

# ----------------------------- Get Files Path ------------------------------
UI_file_name = 'newportgui_v1.1.ui'
current_file_dir = pathlib.Path(__file__).parent.absolute()
UI_dir = os.path.join(current_file_dir, UI_file_name)
icon_dir = os.path.join(current_file_dir, "icon", 'moon_phase_full.ico')

class MainWindow(QtWidgets.QMainWindow):
    def __init__(self, *args, **kwargs):
        QtWidgets.QMainWindow.__init__(self)
        self.setObjectName("BOA BOA BOA")
        self.ui = uic.loadUi(UI_dir, self)
        
        self.resize(800, 540)
        icon = QtGui.QIcon()
        icon.addPixmap(QtGui.QPixmap(icon_dir), QtGui.QIcon.Normal,
                       QtGui.QIcon.Off)
        self.setWindowIcon(icon)
        
        #self.actionQuit.triggered.connect(lambda: self.exitEvent)
        
        self.thread = {}
        self.statusBar().showMessage('Ready')
        
        self.pbMotorJogGo.setEnabled(True)
        
        
        ### Motor init
        self.motordirection = None
        self.init_motor()
        
        
    def closeEvent(self, event):
        reply = QtWidgets.QMessageBox.question(self, 'Quit', 'Are you sure you want to quit?',
        QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No, QtWidgets.QMessageBox.No)
        if reply == QtWidgets.QMessageBox.Yes:
            event.accept()
        else:
            event.ignore()
            
            
    def exitEvent(self):
        reply = QtWidgets.QMessageBox.question(self, 'Quit', 'Are you sure you want to quit?',
        QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No, QtWidgets.QMessageBox.No)
        if reply == QtWidgets.QMessageBox.Yes:
            sys.exit(app.exec_())
        else:
            pass
        
    def init_motor(self):

        self.plainTextEditMotorConnexion.setReadOnly(True)
        self.MotorStatus = 0
        
        self.pbMotorConnexion.clicked.connect(lambda: self.connectMotor())
        
            
        # Motor / Jog/Cycle tab
        self.pushButtonLeftDir.clicked.connect(lambda: self.setdir("-"))
        self.pushButtonRightDir.clicked.connect(lambda: self.setdir("+"))
        
        # Motor / Setup tab
        self.btnApplyMotorDefault.clicked.connect(lambda: self.setdefaultmotorsetup())
        self.btnApplyMotorApply.clicked.connect(lambda: self.applynewmotorsetup())
        
        
        
    def connectMotor(self):
        """
        Function to connect picomotor
        
        
        """
        try: 
            if self.MotorStatus == 0: 
                self.Motor_idProduct = '0x4000' # '0x4000'
                self.Motor_idVendor = '0x104d'  # '0x104d'

                # convert hex value in string to hex value
                self.Motor_idProduct = int(self.Motor_idProduct, 16)
                self.Motor_idVendor = int(self.Motor_idVendor, 16)

                self.picomotor = Pico8742Ctrl(idProduct = self.Motor_idProduct, 
                                               idVendor = self.Motor_idVendor)

                if self.picomotor.message != "ERROR: Device not found": 

                    self.motorDefaultAcc = self.picomotor.get_acceleration()
                    pos = self.motorDefaultAcc.find(">")
                    self.motorDefaultAcc = self.motorDefaultAcc[pos+1:]
                    self.motorAcc = self.motorDefaultAcc

                    self.motorDefaultVel = self.picomotor.get_velocity()
                    pos = self.motorDefaultVel.find(">")
                    self.motorDefaultVel = self.motorDefaultVel[pos+1:]

                    self.motorDefaultPos = self.picomotor.get_position()
                    pos = self.motorDefaultPos.find(">")
                    self.motorDefaultPos = self.motorDefaultPos[pos+1:]

                    self.lineEditacceleration.setText(self.motorDefaultAcc)
                    self.lineEditvelocity.setText(self.motorDefaultVel)
                    self.lineEditpositionstatus.setText(self.motorDefaultPos)

                    self.plainTextEditMotorConnexion.setPlainText(self.picomotor.message)
                    self.MotorStatus = 1
                    self.statusBar().showMessage('Motor connected')
                    
                    self.pbMotorJogGo.setEnabled(True)
                    self.pbMotorJogGo.clicked.connect(self.motorRunJogStyle())
                    
                    
                else:
                    self.plainTextEditMotorConnexion.setPlainText(self.picomotor.message)
                    self.MotorStatus = 0
                    self.statusBar().showMessage('Motor not connected')
            else:
                self.plainTextEditMotorConnexion.setPlainText(MESSAGEMOTORALREADYCONNECTED)
        except:
            self.plainTextEditMotorConnexion.setPlainText(MESSAGEMOTORPERMISSIONERROR)

    def setdir(self, direction):
        
        if direction == "-":
            self.plainTextEditMotorConnexion.setPlainText("Motor setup : anti clockwise direction")
            self.pushButtonLeftDir.setStyleSheet("background-color: green")
            self.pushButtonRightDir.setStyleSheet("background-color: none")
            self.motordirection = "-"

        else:
            self.plainTextEditMotorConnexion.setPlainText("Motor setup : clockwise direction")
            self.pushButtonRightDir.setStyleSheet("background-color: green")
            self.pushButtonLeftDir.setStyleSheet("background-color: none")
            self.motordirection = "+"
    
    
    def setdefaultmotorsetup(self):
        """
        Function to set position, acceleration 
        and velocity to the defaults values. 
        
        """
        reply = QtWidgets.QMessageBox.question(self, 
                        'Set to default values', 'Are you sure ?',
        QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No, 
                                QtWidgets.QMessageBox.No)
        if reply == QtWidgets.QMessageBox.Yes:
            self.lineEditacceleration.setValidator(QtGui.QIntValidator())
            self.lineEditacceleration.setText(self.motorDefaultAcc)
            self.motorAcc = self.motorDefaultAcc
            self.lineEditvelocity.setValidator(QtGui.QIntValidator())
            self.lineEditvelocity.setText(self.motorDefaultVel)
            self.motorVel = self.motorDefaultVel
            self.lineEditpositionstatus.setValidator(QtGui.QIntValidator())
            self.lineEditpositionstatus.setText(self.motorDefaultPos)
            
            self.motorPos = self.motorDefaultPos
            
        else:
            pass
        
        
    def applynewmotorsetup(self):
        """
        Function to set position, acceleration 
        and velocity with some new values. 
        
        """
        reply = QtWidgets.QMessageBox.question(self, 'Save the new values', 'Are you sure ?',
        QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No, 
                            QtWidgets.QMessageBox.No)
        if reply == QtWidgets.QMessageBox.Yes:
            self.lineEditacceleration.setValidator(QtGui.QIntValidator())
            #self.lineEditacceleration.setText(self.motorDefaultAcc)
            self.motorAcc = self.lineEditacceleration.text()
            print(f"New value of acceleration : {self.lineEditacceleration.text()}")
            self.lineEditvelocity.setValidator(QtGui.QIntValidator())
            #self.lineEditvelocity.setText(self.motorDefaultVel)
            self.motorVel = self.lineEditvelocity.text()
            print(f"New value of velocity : {self.lineEditvelocity.text()}")
            self.lineEditpositionstatus.setValidator(QtGui.QIntValidator())
            #self.lineEditpositionstatus.setText(self.motorDefaultPos)
            self.motorPos = self.lineEditpositionstatus.text()
            print(f"New value of position : {self.lineEditpositionstatus.text()}")
            
        else:
            pass
    
    
    def setdefaultstatus(self):
        """
        Function to set position, acceleration and velocity to the 
        defaults values. 
        
        """
        reply = QtWidgets.QMessageBox.question(self, 'Set to default values', 'Are you sure ?',
        QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No, QtWidgets.QMessageBox.No)
        if reply == QtWidgets.QMessageBox.Yes:
            self.lineEditacceleration.setText(self.motorDefaultAcc)
            self.motorAcc = self.motorDefaultAcc
            self.lineEditvelocity.setText(self.motorDefaultVel)
            self.motorVel = self.motorDefaultVel
            self.lineEditpositionstatus.setText(self.motorDefaultPos)
            self.motorPos = self.motorDefaultPos
            
        else:
            pass

        
    def motorRunJogStyle(self):
        
        if self.motordirection is not None:
            print(self.motordirection)
        
        else:
            reply = QtWidgets.QMessageBox.Warning(self, "Not direction selected", "Please select a direction of the motor first", QMessageBox.Ok)
            if reply == QtWidgets.QMessageBox.Ok:
                pass
            
            
            
        
        
        
        
        
        
    def start_worker_1(self):
        self.thread[1] = ThreadClass(parent=None, index=1)
        self.thread[1].start()
        self.thread[1].any_signal.connect(self.my_function)
        self.pushButton.setEnabled(False)

    

    def stop_worker_1(self):
        self.thread[1].stop()
        self.pushButton.setEnabled(True)

    def stop_worker_2(self):
        self.thread[2].stop()
        self.pushButton_2.setEnabled(True)
        
    def my_function(self, counter):
        pass


class ThreadClass(QtCore.QThread):

    any_signal = QtCore.pyqtSignal(int)

    def __init__(self, parent=None, index=0):
        super(ThreadClass, self).__init__(parent)
        self.index = index
        self.is_running = True

    def run(self):
        print('Starting thread...', self.index)
        cnt = 0
        while (True):
            cnt += 1
            if cnt == 99:
                cnt = 0
            time.sleep(0.01)
            self.any_signal.emit(cnt)

    def stop(self):
        self.is_running = False
        print('Stopping thread...', self.index)
        self.terminate()

app = QtWidgets.QApplication(sys.argv)
mainWindow = MainWindow()
mainWindow.show()
sys.exit(app.exec_())


Overwriting guipioneers.py


In [ ]:
%run guipioneers.py